# That backend was tested on Colab with free GPU setup.

##Setup

In [ ]:
!pip install codecarbon flask_cors
!pip install --pre timm
!mkdir '/content/solutions'
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
!gunzip imagenette2-320.tgz
!tar -xvf imagenette2-320.tar
!wget https://raw.githubusercontent.com/raghakot/keras-vis/master/resources/imagenet_class_index.json
from timm.data.dataset import ImageDataset
from PIL import Image
dataset = ImageDataset('./imagenette2-320/val')

## Flask server with submission endpoint

In [37]:
%%writefile flask_app.py 
import pandas as pd
import urllib
import urllib.request
from flask_cors import CORS, cross_origin
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
from codecarbon import EmissionsTracker
from sklearn.metrics import accuracy_score
import numpy as np
import gc
import torch
import os
import os.path
import sys
import json
import imp

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'



@app.route("/")
def home():
    return "Hello World"


@app.route("/submission", methods=['POST'])
@cross_origin()
def submission():
    f = request.files['file']
    name_solution = request.args.get("name_solution")
    fname = os.path.join('/content/solutions', secure_filename(f.filename))
    f.save(fname)
    torch.cuda.empty_cache()
    gc.collect()
    os.rename(fname, '/content/solutions/sol.py')
    sys.path.insert(0, '/content/solutions')
    import sol
    imp.reload(sol)
    with EmissionsTracker(project_name = name_solution) as tracker:
        df_pred_name = sol.check_soluiton('test_submission.csv')
    df_pred = pd.read_csv('predict_submission.csv')
    final_acc = accuracy_score(df_pred.predictions, df_pred.true_labels)
    df_emi = pd.read_csv('emissions.csv')
    if not os.path.exists('acc.npy'):
        np.save('acc.npy', np.array(final_acc))
    else:
        acc_arr = np.load('acc.npy')
        np.save('acc.npy', np.append([acc_arr], final_acc))
    acc_arr = np.load('acc.npy')
    df_emi['accuracy'] = acc_arr
    #df_emi.iloc[-1, df_emi.columns.get_loc('accuracy')] = final_acc
    df_emi.to_csv('emissions_base.csv',index=False)
    read_csv = pd.read_csv('emissions_base.csv',delimiter= ',') # or delimiter = ';'
    csv_to_json = read_csv.T.to_json(orient = 'columns')
    torch.cuda.empty_cache()
    gc.collect()
    return json.loads(csv_to_json)



@app.route("/get_json", methods=['POST'])
@cross_origin()
def get_json():
    if os.path.exists('emissions_base.csv'):
        read_csv = pd.read_csv('emissions_base.csv',delimiter= ',') # or delimiter = ';'
        csv_to_json = read_csv.T.to_json(orient = 'columns')
        return json.loads(csv_to_json)
    else:
        return json.loads('{}'), 200



if __name__ == '__main__':
      app.run(host='0.0.0.0', port=8518)

Writing flask_app.py


In [ ]:
!python /content/flask_app.py & npx localtunnel --port 8518

/content/flask_app.py:16: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
 * Serving Flask app 'flask_app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8518
 * Running on http://172.28.0.12:8518
Press CTRL+C to quit
npx: installed 22 in 5.16s
your url is: https://grumpy-schools-swim-34-83-145-229.loca.lt
127.0.0.1 - - [24/Mar/2023 00:40:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 00:41:36] "OPTIONS /get_json HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 00:41:37] "POST /get_json HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 00:41:41] "POST /get_json HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 00:41:46] "OPTIONS /get_json HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 00:41:46] "POST /get_json HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 00:41:51] "POST /get_json HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2023 00:41:56] "OPTIONS /get_json HTTP/1.1" 200 -
127.0.0.1

## Submission script - allows to evaluate model with provided (Imagenet 1k) Dataset

In [21]:
%%writefile solutions/sol.py

def compute_batch(photos_batch, transform, model):
    from PIL import Image
    import torch
    # Load all the photos from the files
    photos = [Image.open(photo_file).convert('RGB') for photo_file in photos_batch]
    
    # Preprocess all photos
    photos_preprocessed = torch.stack([transform(photo) for photo in photos]).cuda()

    with torch.no_grad():
        # Encode the photos batch to compute the feature vectors and normalize them
        output = model(photos_preprocessed)
        top1_probabilities, top1_class_indices = torch.topk(output.softmax(dim=1) * 100, k=1)

    # Transfer the feature vectors back to the CPU and convert to numpy
    return top1_class_indices

def check_soluiton(input_df_name):
    import pandas as pd
    df = pd.read_csv(input_df_name)
    import timm 
    import torch
    import math
    import numpy as np
    from timm.data.transforms_factory import create_transform
    from timm.data import resolve_data_config

    model = timm.create_model('convnext_nano.in12k_ft_in1k', pretrained = True).cuda()
    model.eval()
    transform = create_transform(**resolve_data_config(model.pretrained_cfg, model=model))
    l_data = df.imgs_path
    batch_size = 64
    # Compute how many batches are needed
    batches = math.ceil(len(l_data) / batch_size)
    pred_l = []

    for i in range(batches):
        batch_files = l_data[i*batch_size : (i+1)*batch_size]
        batch_probas = compute_batch(batch_files, transform, model)
        pred_l.append(batch_probas.cpu())
    df['predictions'] = np.vstack(pred_l)
    df.to_csv('predict_submission.csv')

Overwriting solutions/sol.py


## Test the solution locally, without submittion from frontend side

In [22]:
import pandas as pd
import os.path
from codecarbon import EmissionsTracker
from sklearn.metrics import accuracy_score
import numpy as np
import sys
import imp
sys.path.insert(0, '/content/solutions')
import sol
imp.reload(sol)
with EmissionsTracker(project_name = "convnext_nano.in12k_ft_in1k") as tracker:
    df_pred_name = sol.check_soluiton('test_submission.csv')
df_pred = pd.read_csv('predict_submission.csv')
final_acc = accuracy_score(df_pred.predictions, df_pred.true_labels)
df_emi = pd.read_csv('emissions.csv')
if not os.path.exists('acc.npy'):
    np.save('acc.npy', np.array(final_acc))
else:
    acc_arr = np.load('acc.npy')
    np.save('acc.npy', np.append([acc_arr], final_acc))
acc_arr = np.load('acc.npy')
df_emi['accuracy'] = acc_arr
#df_emi.iloc[-1, df_emi.columns.get_loc('accuracy')] = final_acc
df_emi.to_csv('emissions_base.csv',index=False)

[codecarbon INFO @ 00:28:28] [setup] RAM Tracking...
[codecarbon INFO @ 00:28:28] [setup] GPU Tracking...
[codecarbon INFO @ 00:28:28] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 00:28:28] [setup] CPU Tracking...
[codecarbon WARNING @ 00:28:28] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 00:28:29] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 00:28:29] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 00:28:29] >>> Tracker's metadata:
[codecarbon INFO @ 00:28:29]   Platform system: Linux-5.10.147+-x86_64-with-glibc2.31
[codecarbon INFO @ 00:28:29]   Python version: 3.9.16
[codecarbon INFO @ 00:28:29]   Available RAM : 12.681 GB
[codecarbon INFO @ 00:28:29]   CPU count: 2
[codecarbon INFO @ 00:28:29]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 00:28:29]   GPU count: 1
[codecarbon INFO @ 00:28:29]   GPU model: 1 x Te

[codecarbon INFO @ 00:28:44] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.755449295043945 W
[codecarbon INFO @ 00:28:44] Energy consumed for all GPUs : 0.000270 kWh. All GPUs Power : 64.86600000000001 W
[codecarbon INFO @ 00:28:44] Energy consumed for all CPUs : 0.000177 kWh. All CPUs Power : 42.5 W
[codecarbon INFO @ 00:28:44] 0.000468 kWh of electricity used since the begining.
[codecarbon INFO @ 00:28:59] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.755449295043945 W
[codecarbon INFO @ 00:28:59] Energy consumed for all GPUs : 0.000437 kWh. All GPUs Power : 39.974000000000004 W
[codecarbon INFO @ 00:28:59] Energy consumed for all CPUs : 0.000354 kWh. All CPUs Power : 42.5 W
[codecarbon INFO @ 00:28:59] 0.000830 kWh of electricity used since the begining.
[codecarbon INFO @ 00:29:13] Energy consumed for RAM : 0.000058 kWh. RAM Power : 4.755449295043945 W
[codecarbon INFO @ 00:29:13] Energy consumed for all GPUs : 0.000755 kWh. All GPUs Power : 82.757 W
[codecarbon IN

In [ ]:
#3 - maxxvitv2_rmlp_base_rw_224.sw_in12k_ft_in1k
#4 - coatnet_rmlp_1_rw2_224.sw_in12k_ft_in1k
#5 - coatnet_rmlp_nano_rw_224.sw_in1k
#6 - maxvit_rmlp_pico_rw_256.sw_in1k


In [25]:
df

,timestamp,project_name,run_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,cpu_energy,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,accuracy
0,2023-03-23T20:06:58,maxvit_tiny_tf_224.in1k,9b83cbb9-f023-473b-8086-3d67a8e61df5,65.549083,0.000404,0.006171,42.5,74.621,4.755446,0.000774,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-77.0365,38.8940,12.681190,machine,N,0.880000
1,2023-03-23T20:11:45,convnext_large_mlp.clip_laion2b_augreg_ft_in1k,2d71b961-c813-4dca-b4ff-3addd25fa4d8,213.521032,0.001404,0.006576,42.5,78.517,4.755446,0.002521,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-77.0365,38.8940,12.681190,machine,N,0.933503
2,2023-03-23T23:29:18,maxxvitv2_rmlp_base_rw_224.sw_in12k_ft_in1k,dd4369c6-c326-4ab6-8f5d-29bf1d018401,147.855654,0.000615,0.004159,42.5,50.971,4.755449,0.001745,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-121.1871,45.5999,12.681198,machine,N,0.903185
3,2023-03-23T23:49:28,coatnet_rmlp_1_rw2_224.sw_in12k_ft_in1k,12880bba-88b7-4b9f-ba2e-abfd2243bb72,70.037551,0.000269,0.003846,42.5,66.366,4.755449,0.000826,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-121.1871,45.5999,12.681198,machine,N,0.888408
4,2023-03-23T23:51:12,coatnet_rmlp_nano_rw_224.sw_in1k,b6ec2aa0-1850-411e-aa97-1efdb9638fea,45.517394,0.000160,0.003506,42.5,79.681,4.755449,0.000537,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-121.1871,45.5999,12.681198,machine,N,0.872866
5,2023-03-24T00:20:22,maxvit_rmlp_pico_rw_256.sw_in1k,f14df9e6-4cad-4c25-84ec-d439b99e4b54,46.287312,0.000172,0.003709,42.5,83.238,4.755449,0.000546,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-121.1871,45.5999,12.681198,machine,N,0.848153
6,2023-03-24T00:22:57,vit_base_patch32_clip_224.laion2b_ft_in1k,230bcc24-b18a-45f5-abaa-e44efea8bb28,45.179335,0.000183,0.004057,42.5,119.814,4.755449,0.000533,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-121.1871,45.5999,12.681198,machine,N,0.887643
7,2023-03-24T00:24:46,maxvit_small_tf_224.in1k,a0eb283e-8060-4f1b-a630-a8ad0818e7e0,82.980016,0.000370,0.004455,42.5,70.640,4.755449,0.000979,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-121.1871,45.5999,12.681198,machine,N,0.902166
8,2023-03-24T00:25:37,mobilenetv2_050,5edc63ce-723c-4019-832b-c018dbc85f1a,27.849938,0.000094,0.003363,42.5,28.210,4.755449,0.000329,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-121.1871,45.5999,12.681198,machine,N,0.745478
9,2023-03-24T00:28:16,convnext_base.clip_laion2b_augreg_ft_in1k,150e7626-1cfd-4a87-957d-9f9a3ae9252e,125.562139,0.000488,0.003890,42.5,53.513,4.755449,0.001482,...,2,Intel(R) Xeon(R) CPU @ 2.20GHz,1,1 x Tesla T4,-121.1871,45.5999,12.681198,machine,N,0.930955
